In [ ]:
pip install mediapipe

In [ ]:
pip install pymysql

In [ ]:
pip install --user Flask urllib3 pymysql mediapipe opencv-python numpy matplotlib requests Pillow

In [ ]:
pip install opencv-python opencv-contrib-python

In [ ]:
# Flask 서버에서 구동
# 스프링 http://59.0.234.211:8087/pytest 와 Flask http://59.0.234.211:5000/testpage 연동

from flask import Flask, request
import urllib.request
import pymysql
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import requests
from PIL import Image, ImageDraw, ImageOps

app = Flask(__name__)

@app.route("/pytest", methods=['GET'])
def test() :
    salonid = request.args.get('salon_id')
    
    model_path = "hair_segmenter.tflite"
    # Height and width that will be used by the model
    DESIRED_HEIGHT = 480
    DESIRED_WIDTH = 480
    def resize_and_show(image):
        h, w = image.shape[:2]
        if h < w:
            img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
        else:
            img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
    
    # 미용실 id 설정
    salon_id = salonid
    afterImg = ''
    hair_id = ''
    
    # db연결, sql문 작성
    conn = pymysql.connect(host='project-db-cgi.smhrd.com',port=3307, user='campus_b_230519_3',password='smhrd3',db='campus_b_230519_3',charset='utf8')
    cur = conn.cursor()
    hair_id=''
    
    # 결과값의 마지막 값 선택
    sql = f"select pic_path from my_history where salon_id = '{salon_id}'"   
    cur.execute(sql)
    result = cur.fetchall()
    afterImg = result[-1][0]
    
    #with를 사용하면 바로 close 되어서 아래 쿼리 작동이 안 됨
    #with conn:
    #     with conn.cursor() as cur:
    #        cur.execute(sql,(salon_id))
    #        result = cur.fetchall()
    #        afterImg = result[-1][0]
    #        print(afterImg)

    # 이미지 불러올 폴더
    IMAGE_FOLDER = 'C:/Users/user/git/final_project_t3/SmartMirror/src/main/resources/static/afterImg'

    IMAGE_FILENAMES = [afterImg]

    for name in IMAGE_FILENAMES:
        image_path = os.path.join(IMAGE_FOLDER, name)+'.jpg'

    # Performs resizing and showing the image
    image_path=image_path
    images = {name: cv2.imread(image_path) for name in IMAGE_FILENAMES}
    images.items()

    BG_COLOR = (192, 192, 192) # gray
    MASK_COLOR = (255, 255, 255) # white

    # options
    base_options = python.BaseOptions(model_asset_path=model_path)
    options = vision.ImageSegmenterOptions(base_options=base_options,
                                           output_category_mask=True)

    # 세그먼트 샘플링
    # segmenter 생성
    with vision.ImageSegmenter.create_from_options(options) as segmenter:

      # 이미지폴더를 순회하며 이미지파일을 읽어옴
        for name in IMAGE_FILENAMES:
            image_file_path = os.path.join(IMAGE_FOLDER, name)

          # Create the MediaPipe image file that will be segmented
            image = mp.Image.create_from_file(image_file_path)

          # Retrieve the masks for the segmented image
            segmentation_result = segmenter.segment(image)
            category_mask = segmentation_result.category_mask

          # Generate solid color images for showing the output segmentation mask.
            image_data = image.numpy_view()
            fg_image = np.zeros(image_data.shape, dtype=np.uint8)
            fg_image[:] = MASK_COLOR
            bg_image = np.zeros(image_data.shape, dtype=np.uint8)
            bg_image[:] = BG_COLOR

            condition = np.stack((category_mask.numpy_view(),) * 3, axis=-1) > 0.2
            output_image = np.where(condition, fg_image, bg_image)

    # 헤어 세그먼트(머리카락만 추출)


    with vision.ImageSegmenter.create_from_options(options) as segmenter:

      # Loop through demo image(s)
        for name in IMAGE_FILENAMES:
            image_file_path = os.path.join(IMAGE_FOLDER, name)

          # Create the MediaPipe image file that will be segmented
            image = mp.Image.create_from_file(image_file_path)

          # Retrieve the masks for the segmented image
            segmentation_result = segmenter.segment(image)
            category_mask = segmentation_result.category_mask

            image_data = cv2.cvtColor(image.numpy_view(), cv2.COLOR_BGR2RGB)
          # category_mask 를 넘파이 배열로 변환
            category_mask = category_mask.numpy_view()

          # Apply hair mask to the original image
            output_image = cv2.bitwise_and(image_data, image_data, mask=category_mask)

          # Create a mask for the black background
            black_bg_mask = np.all(output_image == [0, 0, 0], axis=2)

          # Create an alpha channel with the black background mask
            alpha_channel = np.where(black_bg_mask, 0, 255).astype(np.uint8)

          # Add the alpha channel to the hair image
            hair_image_with_alpha = cv2.cvtColor(output_image, cv2.COLOR_BGR2BGRA)
            hair_image_with_alpha[:, :, 3] = alpha_channel

          # 헤어 스타일 이미지 테이블 시퀀스 값 조회 후 가장 마지막 값 선택하여 다음 값을 파일 이름에 더해준다.
            sql2 = "select sequence from hair_style_img"
            seq = ''
   
            cur.execute(sql2)
            result = cur.fetchall()
            seq = result[-1][0]+1
            seq = str(seq)
            print('sequence : '+str(seq))
            
          # 헤어 스타일 이미지 파일명
            hair_id = 'hairStyle_'+salon_id+'_'+seq
          
          # 헤어 스타일 이미지 저장 경로
            output_path = 'C:/Users/user/git/final_project_t3/SmartMirror/src/main/resources/static/hairStyle/'+hair_id+'.jpg'
            cv2.imwrite(output_path, hair_image_with_alpha)
          
          # 헤어 스타일 이미지 DB 저장
            sql3 = f"insert into hair_style_img (hair_id, salon_id) values ('{hair_id}','{salon_id}')"
            cur.execute(sql3)
            conn.commit()
        
          # DB 접속 종료
            conn.close()
          # Show the image with hair segmentation
          # resize_and_show(hair_image_with_alpha)
          
          # ---------- 합성 -------------
          # 머리카락 이미지 경로
            hair_image_path = output_path

            # 합성할 원하는 이미지 경로
            mannequin_image_path = "C:/Users/user/git/final_project_t3/SmartMirror/src/main/resources/static/maskmodel/mane5.jpg"

            # 머리카락 이미지 읽기
            hair_image = cv2.imread(hair_image_path)
            hair_image = cv2.cvtColor(hair_image, cv2.COLOR_BGR2RGB)

            # 합성할 원하는 이미지 읽기
            mannequin_image = cv2.imread(mannequin_image_path)
            mannequin_image = cv2.cvtColor(mannequin_image, cv2.COLOR_BGR2RGB)

            # 머리카락 이미지 크기에 맞게 원하는 이미지 크기 조정
            mannequin_resized = cv2.resize(mannequin_image, (hair_image.shape[1], hair_image.shape[0]))

            # 머리카락 이미지를 위한 마스크 생성
            mask = np.zeros_like(hair_image)
            mask[np.where((hair_image > [0, 0, 0]).all(axis=2))] = [255, 255, 255]  # 머리카락 부분은 흰색으로 마스킹

            # 마스크를 이용하여 머리카락 부분만 추출
            hair_extracted = cv2.bitwise_and(hair_image, mask)

            # 추출한 머리카락 이미지와 합성할 원하는 이미지 합성
            result = cv2.bitwise_or(hair_extracted, mannequin_resized)

            # 알파 채널 생성
            alpha_channel = np.zeros((hair_image.shape[0], hair_image.shape[1]), dtype=np.uint8)
            alpha_channel[np.where((hair_extracted > [0, 0, 0]).all(axis=2))] = 255  # 머리카락 부분은 완전히 불투명하게

            # RGBA 이미지 생성
            rgba_image = cv2.cvtColor(hair_extracted, cv2.COLOR_RGB2RGBA)
            rgba_image[:, :, 3] = alpha_channel  # 알파 채널 설정

            # RGBA이미지를 PIL이미지로 변환
            pil_image = Image.fromarray(rgba_image)

            # pil이미지를 RGBA모드로 변환
            pil_image_rgba = pil_image.convert("RGBA")

            # mannequin_resized를 PIL Image로 변환한 다음 RGBA 모드로 변환
            mannequin_pil = Image.fromarray(mannequin_resized).convert("RGBA")

            # 이미지 합성
            composite_image = Image.alpha_composite(mannequin_pil, pil_image_rgba)

            # composite_image를 다시 numpy 배열로 변환하는 작업을 의미합니다.
            composite_array = np.array(composite_image)

            # 흰색 픽셀을 검정색으로 변환할 범위를 정의
            white_range = (composite_array >= [250, 250, 250, 255]).all(axis=2) & (composite_array <= [255, 255, 255, 255]).all(axis=2)

            # 흰 픽셀을 검정색으로 변환
            composite_array[white_range] = [0, 0, 0, 255]

            # 합성된 이미지를 디스플레이에 출력
            #plt.imshow(composite_array)
            #plt.axis('off')
            #plt.show()

            # 이미지 파일 경로
            output_path = 'C:/Users/user/git/final_project_t3/SmartMirror/src/main/resources/static/maskhairstyle/'+hair_id+'.jpg'

            # BGR을 RGB로 저장
            composite_array = cv2.cvtColor(composite_array, cv2.COLOR_BGR2RGB)

            # composite_array 이미지 저장
            cv2.imwrite(output_path, composite_array)

    return 'success!!'

if __name__ == '__main__':
    app.run(debug=False,host="localhost",port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5001
Press CTRL+C to quit


sequence : 35


127.0.0.1 - - [08/Jun/2023 10:20:12] "GET /pytest?salon_id=a000 HTTP/1.1" 200 -


In [1]:
from flask import Flask, request

app = Flask(__name__)

@app.route('/pytest', methods=['GET'])
def process_query_param():
    # 쿼리 파라미터 'word'를 받음
    word = request.args.get('salon_id')

    # 받은 단어를 가공 또는 처리하는 로직을 추가
    # TODO: 단어를 원하는 방식으로 가공 또는 처리

    # 가공된 결과를 반환
    return 'Salon_id : {}'.format(word)

if __name__ == '__main__':
    app.run(host='localhost', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5001
Press CTRL+C to quit
127.0.0.1 - - [08/Jun/2023 10:15:18] "GET /pytest?salon_id=a000 HTTP/1.1" 200 -


In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/pytest', methods=['GET'])
def get_string():
    return "Hello, Flask!"

if __name__ == '__main__':
    app.run(host='localhost', port=5000)